https://docs.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/quickstarts/image-classification?pivots=programming-language-python

In [3]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry
from msrest.authentication import ApiKeyCredentials

ENDPOINT = "https://soybean-classifier.cognitiveservices.azure.com/"

# Replace with a valid key
training_key = !az cognitiveservices account keys list -n soybean-classifier -g soybean-classifier --query 'key1' -o tsv
prediction_keys = !az cognitiveservices account keys list -n soybeanclassifier-Prediction -g soybean-classifier --query 'key1' -o tsv
prediction_resource_id = "/subscriptions/8a8d5e01-2bfc-417c-83c6-1766d8a6533e/resourceGroups/soybean-classifier/providers/Microsoft.CognitiveServices/accounts/soybeanclassifier-Prediction"

publish_iteration_name = "abo-classification"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key[0]})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

In [15]:
for project in trainer.get_projects():
    print("{:>20}    {:30}".format(project.name, project.id))

 apple-banana-orange    067b25c1-b86d-47d9-8719-0f9c5e3c6039
  soybean-classifier    4e50a55c-8ed0-47f5-a65e-2b96aee37329


## DO NOT EXECUTE

In [ ]:
# Create a new project
print ("Creating project...")
project = trainer.create_project("apple-banana-orange")

In [ ]:
# Make two tags in the new project
hemlock_tag = trainer.create_tag(project.id, "Hemlock")
cherry_tag = trainer.create_tag(project.id, "Japanese Cherry")

In [ ]:
base_image_url = "<path to repo directory>/cognitive-services-python-sdk-samples/samples/vision/"

print("Adding images...")

image_list = []

for image_num in range(1, 11):
    file_name = "hemlock_{}.jpg".format(image_num)
    with open(base_image_url + "images/Hemlock/" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[hemlock_tag.id]))

for image_num in range(1, 11):
    file_name = "japanese_cherry_{}.jpg".format(image_num)
    with open(base_image_url + "images/Japanese Cherry/" + file_name, "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[cherry_tag.id]))

upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

## Train the classifier and publish

In [ ]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

In [10]:
# The iteration is now trained. Publish it to the project endpoint
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


## Get and use the published iteration on the prediction endpoint

In [8]:
project = trainer.get_project('067b25c1-b86d-47d9-8719-0f9c5e3c6039')

In [9]:
iteration = trainer.get_iteration('067b25c1-b86d-47d9-8719-0f9c5e3c6039','1ac14df9-59a9-4876-a56f-403661204f03')

In [12]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

base_image_url = '/home/neophilex/learn_azure/'
# Now there is a trained endpoint that can be used to make a prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_keys[0]})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

with open(base_image_url + "test.jpg", "rb") as image_contents:
    results = predictor.classify_image(
        project.id, publish_iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print("{:>20}\t".format(prediction.tag_name) +
              ": {0:.2f}%".format(prediction.probability * 100))

              orange	: 99.79%
       rotten orange	: 0.21%
               apple	: 0.00%
              banana	: 0.00%
         green apple	: 0.00%
        rotten apple	: 0.00%
       rotten banana	: 0.00%


In [ ]:
prediction_keys[0]